# 

In [1]:
df = pd.read_csv('../MACH_data/data_cutoff_age18.csv' )

In [2]:
df = df.loc[:,["score", "v_score", "t_score", "m_score", "voted"]]
df

,score,v_score,t_score,m_score,voted
0,75.0,35.0,33.0,7.0,Yes
1,91.0,37.0,45.0,9.0,No
2,78.0,37.0,36.0,5.0,Yes
3,85.0,35.0,41.0,9.0,Yes
4,66.0,28.0,31.0,7.0,Yes
...,...,...,...,...,...
54713,59.0,28.0,26.0,5.0,No
54714,90.0,37.0,43.0,10.0,No
54715,69.0,34.0,27.0,8.0,No
54716,53.0,25.0,22.0,6.0,Yes


In [3]:
y = df["voted"]
x = df.drop("voted", axis=1)

In [5]:
from sklearn.preprocessing import (LabelEncoder, LabelBinarizer, RobustScaler)

In [6]:
lb = LabelBinarizer()
Y = lb.fit_transform(y)

In [7]:
X = pd.get_dummies(x)
X

,score,v_score,t_score,m_score
0,75.0,35.0,33.0,7.0
1,91.0,37.0,45.0,9.0
2,78.0,37.0,36.0,5.0
3,85.0,35.0,41.0,9.0
4,66.0,28.0,31.0,7.0
...,...,...,...,...
54713,59.0,28.0,26.0,5.0
54714,90.0,37.0,43.0,10.0
54715,69.0,34.0,27.0,8.0
54716,53.0,25.0,22.0,6.0


In [9]:
Y

array([[1],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=13)

### 모델 돌리기

In [11]:
# ada, gbc, xgb, lgbm

In [20]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [24]:
ada = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
xgb = XGBClassifier()
lgbm = LGBMClassifier()

In [23]:
models = [ada, gbc, xgb, lgbm]
model_names = ['Ada', 'GBC', 'XGB', 'LGBM']

In [15]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix)

def get_score(y_test, pred):
    acc = accuracy_score(y_test, pred)
    pre = precision_score(y_test, pred)
    rec = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
   
    return acc, auc, pre, rec, f1

def print_score(y_test, pred, confusion=False, score=False):
    acc, auc, pre, rec, f1 = get_score(y_test, pred)
    con = confusion_matrix(y_test, pred)
    if confusion==True:
        print('confusion matrix')
        print(con)
        print('='*20)
    
    if score==True:
        print('Accuracy: {0:.4f}, AUC: {1:.4f}'.format(acc, auc))
        print('Recall: {0:.4f}, f1_score: {1:.4f}, precision: {2:.4f}'.format(rec, f1, pre))
        

In [16]:
import time

def fit_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pre_tr = model.predict(X_train)
    y_pre_test = model.predict(X_test)
    
    return get_score(y_test, y_pre_test)

def models_score_df(models, model_names, X_train, X_test, y_train, y_test):
    cols_names = ['accuracy', 'AUC', 'precision', 'recall', 'f1']
    datas = []
    for model in models:
        datas.append(fit_model(model, X_train, X_test, y_train, y_test))
     
    return pd.DataFrame(datas, columns=cols_names, index=model_names)


In [27]:
# 숫자 데이터만 robust 

In [25]:
models_score_df(models, model_names, X_train, X_test, y_train, y_test)

,accuracy,AUC,precision,recall,f1
Ada,0.537646,0.527545,0.546234,0.718799,0.620747
GBC,0.539291,0.527027,0.544775,0.759243,0.634373
XGB,0.529788,0.522482,0.543935,0.660823,0.596708
LGBM,0.527412,0.518493,0.540172,0.687381,0.604950
